<a href="https://colab.research.google.com/github/manns01/projects/blob/main/Class_Neutrinos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will work out the effect of neutrinos on the main cosmological observables, the BAO, the matter power spectrum and the CMB.
**This is a shared notebook. Please copy it and run it on google colab or your own laptop!**


In [ ]:
import sys, platform, os
import matplotlib.pyplot as plt
import numpy as np
import scipy

In [ ]:
!pip install classy

- Installing `classy` via pip has been enabled since v3, though is not 100% reliable yet (especially for Mac)


In [ ]:
import classy
print('Using CLASS %s installed at %s'%(classy.__version__,os.path.dirname(classy.__file__)))


# Step 1 Compute the CMB and matter power spectrum in a universe with massless neutrinos. This will be our baseline.


In [ ]:

# create instance of the class "Class"
LambdaCDM = classy.Class()
# pass input parameters (from PlanckTTTEEE+lowE+lensing 1807.06209)
LambdaCDM.set({'omega_b':0.02237,'omega_cdm':0.1200,'h':0.6736,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544})
LambdaCDM.set({'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})

# run class
LambdaCDM.compute()


# In[ ]:

Let's extract some basic background quantities

In [ ]:
bg=LambdaCDM.get_background()
bg.keys() ##let see all the things we can print

Let us, for example, plot energy densities

In [ ]:

opz = 1.+bg['z']
plt.loglog(opz,bg['(.)rho_g']/opz**4,label="photons")
plt.loglog(opz,(bg['(.)rho_b']+bg['(.)rho_cdm'])/opz**4,label="matter")
plt.loglog(opz,bg['(.)rho_lambda']/opz**4,label=r"$\Lambda$")
plt.loglog(opz,bg['(.)rho_ur']/opz**4,label="massless neutrinos")
plt.legend()
plt.xlabel(r"Redshift 1+$z$")
plt.ylabel(r"$8\pi G/3 \times \rho~/~(1+z)^4$ [Mpc$^{-2}$]")
plt.show()

To better see the contribution from neutrinos let's plot ratios

In [ ]:
opz = 1.+bg['z']
plt.semilogx(opz,bg['(.)rho_g']/bg['(.)rho_crit'],label="photons")
plt.semilogx(opz,(bg['(.)rho_cdm']+bg['(.)rho_b'])/bg['(.)rho_crit'],label="matter")
plt.semilogx(opz,bg['(.)rho_lambda']/bg['(.)rho_crit'],label=r'$\Lambda$')
plt.semilogx(opz,bg['(.)rho_ur']/bg['(.)rho_crit'],label="massless neutrinos")
plt.legend()
plt.xlim(1,1e8)
plt.ylim(0,1)
plt.xlabel(r"Redshift 1+$z$")
plt.ylabel(r"Fractional density $\rho/\rho_\mathrm{crit}$")
plt.show()

Let's extract the CMB power spectrum and compare raw/lensed Cl's.

In [ ]:

# get all C_l output
cls = LambdaCDM.lensed_cl(2500)
cls_raw = LambdaCDM.raw_cl(2500)

# To check the format of cls
cls.keys()


# In[ ]:


ll = cls['ell'][2:]
clTT = cls['tt'][2:]
clEE = cls['ee'][2:]
clPP = cls['pp'][2:]

ll_raw = cls_raw['ell'][2:]
clTT_raw = cls_raw['tt'][2:]
clEE_raw = cls_raw['ee'][2:]

print(clTT_raw)

Let's download Planck TT data to compare our predicted power spectrum. We only download data for l > 30 for simplicty

In [ ]:
import requests

url = "https://irsa.ipac.caltech.edu/data/Planck/release_3/ancillary-data/cosmoparams/COM_PowerSpect_CMB-TT-binned_R3.01.txt"
response = requests.get(url)

# Save the file
with open("COM_PowerSpect_CMB-TT-binned_R3.01.txt", "wb") as f:
    f.write(response.content)

# Load the data into a numpy array
obsdat_binned = np.loadtxt("COM_PowerSpect_CMB-TT-binned_R3.01.txt").T

# Display the first few rows of the array
#print(obsdat_binned[0, :])

Let's plot the data


In [ ]:
import matplotlib.pyplot as plt
from math import pi


# In[ ]:


# plot C_l^TT
Tcmb=2.7255*1e6
temp_factor=(Tcmb**2)
plt.figure(1)
plt.xscale('log');plt.yscale('linear');plt.xlim(2,2500)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$[\ell(\ell+1)/2\pi]  C_\ell^\mathrm{TT}$')
plt.plot(ll,clTT*ll*(ll+1)/2./pi*temp_factor,'r-')

plt.errorbar(obsdat_binned[0], obsdat_binned[1], yerr=[obsdat_binned[2], obsdat_binned[3]], fmt='.')
#plt.xlabel("Column 1")
#lt.ylabel("Column 2")
#plt.title("Plot of Column 1 vs Column 2 with Error Bars")
plt.show()

Now let's compare with and without lensing

In [ ]:
# prompt: plot the ratio of ClTT and ClTT_raw

# plot the ratio of ClTT and ClTT_raw
plt.figure(2)
plt.xscale('linear');plt.yscale('linear');plt.xlim(2,2500)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\Delta C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,raw}$')
plt.plot(ll, clTT / clTT_raw-1, 'b-')
plt.errorbar(obsdat_binned[0], (obsdat_binned[1]-obsdat_binned[4])/obsdat_binned[4], yerr=[obsdat_binned[2]/obsdat_binned[4], obsdat_binned[3]/obsdat_binned[4]], fmt='.', color='blue', ecolor='blue', capsize=3)
plt.show()


##lensing is really important!!


Let's look at the matter power spectrum, a key probe of neutrino physics


In [ ]:

ks = np.geomspace(1e-4,1,num=1000)
pks =  LambdaCDM.get_pk_all(ks,z=0,nonlinear=False)

plt.figure()

plt.loglog(ks,pks,linestyle="solid",label=r'$\Lambda$CDM with massless neutrinos')

plt.xlabel("Wavenumber $k$ [1/Mpc]")
plt.ylabel("Power spectrum $P(k)$ [Mpc${}^3$]")
plt.xlim(1e-4,1)
plt.show()

# Step 2: include massive neutrinos!
#we will do 2 cosmologies, $\sum$ $m_\nu$ = 0.06 eV and 0.6eV.

In [ ]:

common_settings=({'omega_b':0.02237,'omega_cdm':0.1200,'h':0.6736,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544,'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})

##we assume 3 degenerate neutrinos in both cases

mnu006eV = classy.Class()
mnu006eV.set(common_settings)
#mnu006eV.set({'m_ncdm':0.06,'N_ur':2.0307,'N_ncdm':1})
mnu006eV.set({'m_ncdm':0.02,'N_ur':0.0043,'N_ncdm':1,'deg_ncdm':3})

mnu006eV.set({'background_verbose':10})
mnu006eV.compute()

mnu06eV = classy.Class()
mnu06eV.set(common_settings)
mnu06eV.set({'background_verbose':10})
mnu06eV.set({'m_ncdm':0.20,'N_ur':0.0043,'N_ncdm':1,'deg_ncdm':3})
mnu06eV.compute()



Let's plot the energy density of the massive neutrinos

In [ ]:
opz = 1.+bg['z']
bg006=mnu006eV.get_background()
bg06=mnu06eV.get_background()

plt.semilogx(opz,bg['(.)rho_g']/bg['(.)rho_crit'],label="photons")
plt.semilogx(opz,(bg['(.)rho_cdm']+bg['(.)rho_b'])/bg['(.)rho_crit'],label="matter")
plt.semilogx(opz,bg['(.)rho_lambda']/bg['(.)rho_crit'],label=r'$\Lambda$')
plt.semilogx(opz,bg['(.)rho_ur']/bg['(.)rho_crit'],label="massless neutrinos")
plt.semilogx(opz,bg006['(.)rho_ncdm[0]']/bg006['(.)rho_crit'],'k-.',label=r"$\sum m_\nu = 0.06$ eV$")
plt.semilogx(opz,bg06['(.)rho_ncdm[0]']/bg06['(.)rho_crit'],'k--',label=r"$\sum m_\nu = 0.6$ eV$")


plt.legend()
plt.xlim(1,1e5)
plt.ylim(0,0.1) #you can zoom in to see the small neutrino mass
#plt.ylim(0,1)
plt.xlabel(r"Redshift 1+$z$")
plt.ylabel(r"Fractional density $\rho/\rho_\mathrm{crit}$")
plt.show()

Let's look at the BAO


In [ ]:
z_BAO=[0.51000000,0.70600000,0.93400000,1.32100000,1.48400000,2.33]
DM_over_rd =[13.58758434,17.35069094,21.57563956,27.60085612,30.51190063,38.988973961958784]
BAO_errors = [0.167,0.177,0.152,0.318,0.760,0.531]


In [ ]:
# prompt: plot the data DM_over_rd normalized to the LambdaCDM prediction with their error bars

# Calculate DM/rd for LambdaCDM for the given BAO redshifts
DM_over_rd_lcdm = np.zeros_like(z_BAO)
DM_over_rd_mnu006 = np.zeros_like(z_BAO)
DM_over_rd_mnu06 = np.zeros_like(z_BAO)

for i, z in enumerate(z_BAO):
    # Calculate the comoving distance (DM)
    DM_over_rd_lcdm[i] = LambdaCDM.comoving_distance(z) / LambdaCDM.rs_drag()
    DM_over_rd_mnu006[i] = mnu006eV.comoving_distance(z) / mnu006eV.rs_drag()
    DM_over_rd_mnu06[i] = mnu06eV.comoving_distance(z) / mnu06eV.rs_drag()

# Normalize the data to the LambdaCDM prediction
normalized_DM_over_rd = DM_over_rd / DM_over_rd_lcdm
normalized_BAO_errors = BAO_errors / DM_over_rd_lcdm

plt.figure()
plt.errorbar(z_BAO, normalized_DM_over_rd, yerr=normalized_BAO_errors, fmt='o', label='BAO Data (Normalized)')
plt.plot(z_BAO, DM_over_rd_mnu006/DM_over_rd_lcdm,  label=r'$\sum m_\nu = 0.06$ eV')
plt.plot(z_BAO, DM_over_rd_mnu06/DM_over_rd_lcdm,   label=r'$\sum m_\nu = 0.6$ eV')

plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM Prediction') # Add a line at 1 for the LambdaCDM reference
plt.xlabel("Redshift z")
plt.ylabel(r"$(D_M(z)/r_d) / (D_M^{\Lambda CDM}(z)/r_d^{\Lambda CDM})$")
plt.title("BAO Data normalized to $\Lambda$CDM Prediction")
plt.legend()
plt.grid(True)
plt.show()

Let's look at the matter power spectrum and the famous suppression due to neutrino masses


In [ ]:
# prompt: print the ratio of the Pks for mnu006eV and mnu06eV with respect to LambdaCDM

# get P(k) for the different models
ks_mnu006eV = np.geomspace(1e-4, 1, num=1000)
pks_mnu006eV = mnu006eV.get_pk_all(ks_mnu006eV, z=0, nonlinear=False)

ks_mnu06eV = np.geomspace(1e-4, 1, num=1000)
pks_mnu06eV = mnu06eV.get_pk_all(ks_mnu06eV, z=0, nonlinear=False)

ks_LambdaCDM = np.geomspace(1e-4, 1, num=1000)
pks_LambdaCDM = LambdaCDM.get_pk_all(ks_LambdaCDM, z=0, nonlinear=False)

# plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel("Wavenumber $k$ [1/Mpc]")
plt.ylabel(r"$P(k) / P(k)_{\Lambda\mathrm{CDM}}$")

plt.plot(ks_mnu006eV, pks_mnu006eV / pks_LambdaCDM-1, label=r'$\sum m_\nu = 0.06$ eV')
#plt.plot(ks_mnu06eV,-8 * mnu006eV.get_background()['(.)rho_ncdm[0]'][-1] / (mnu006eV.get_background()['(.)rho_cdm'][-1]+mnu06eV.get_background()['(.)rho_b'][-1])*ks_mnu06eV/ks_mnu06eV,'k--')

plt.plot(ks_mnu06eV, pks_mnu06eV / pks_LambdaCDM-1, label=r'$\sum m_\nu = 0.6$ eV')
#plt.plot(ks_mnu06eV,-8 * mnu06eV.get_background()['(.)rho_ncdm[0]'][-1] / (mnu06eV.get_background()['(.)rho_cdm'][-1]+mnu06eV.get_background()['(.)rho_b'][-1])*ks_mnu06eV/ks_mnu06eV,'k--')
plt.legend()
plt.xlim(1e-4, 1)
plt.show()

Now let's look at the impact on the CMB with and without lensing


In [ ]:
# prompt: plot the ratio of the CMB power spectrum for mnu006 eV and mnu06 eV with respect to LambdaCDM

# Extract the CMB power spectra for each model
cls_mnu006eV = mnu006eV.lensed_cl(2500)
cls_mnu06eV = mnu06eV.lensed_cl(2500)
cls_LambdaCDM = LambdaCDM.lensed_cl(2500)

# Extract the TT power spectra
clTT_mnu006eV = cls_mnu006eV['tt'][2:]
clTT_mnu06eV = cls_mnu06eV['tt'][2:]
clTT_LambdaCDM = cls_LambdaCDM['tt'][2:]

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]

# Calculate the ratios
ratio_mnu006eV_LambdaCDM = clTT_mnu006eV / clTT_LambdaCDM
ratio_mnu06eV_LambdaCDM = clTT_mnu06eV / clTT_LambdaCDM

# Plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')

plt.plot(ll, ratio_mnu006eV_LambdaCDM, label=r'$\sum m_\nu = 0.06$ eV')
plt.plot(ll, ratio_mnu06eV_LambdaCDM, label=r'$\sum m_\nu = 0.6$ eV')

plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
plt.title("Ratio of CMB TT Power Spectra relative to $\Lambda$CDM")
plt.show()

# Step 3: use degeneracies in the CMB to compensate the effect of neutrinos



Now we fix the angular scale $\theta_s=1.041$ in the CMB, it has been measured at 0.1% accuracy!


In [ ]:

common_settings=({'omega_b':0.02237,'omega_cdm':0.1200,'theta_s_100':1.04110,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544,'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})

##In the first case, assume 1 massive 2 massless
LambdaCDM = classy.Class()
LambdaCDM.set(common_settings)
LambdaCDM.compute()
##In the first case, assume 1 massive 2 massless
mnu006eV = classy.Class()
mnu006eV.set(common_settings)
#mnu006eV.set({'m_ncdm':0.06,'N_ur':2.0307,'N_ncdm':1})
mnu006eV.set({'m_ncdm':0.02,'N_ur':0.0043,'N_ncdm':1,'deg_ncdm':3})

mnu006eV.compute()

#In the 2nd case, assume 3 degenerate massive neutrinos
mnu06eV = classy.Class()
mnu06eV.set(common_settings)
mnu06eV.set({'m_ncdm':0.20,'N_ur':0.0043,'N_ncdm':1,'deg_ncdm':3})
mnu06eV.compute()



In [ ]:
print("h:",LambdaCDM.h(),mnu006eV.h(),mnu06eV.h())
print("Omega_m:",LambdaCDM.Omega_m(),mnu006eV.Omega_m(),mnu06eV.Omega_m())

#we have exploited the degeneracy with H0 to keep the angular scale of the CMB fixed
#This has also significantly changed Omega_m! Much larger than in LCDM

In [ ]:
# prompt: plot the ratio of the CMB power spectrum for mnu006 eV and mnu06 eV with respect to LambdaCDM

# Extract the CMB power spectra for each model
cls_mnu006eV = mnu006eV.lensed_cl(2500)
cls_mnu06eV = mnu06eV.lensed_cl(2500)
cls_LambdaCDM = LambdaCDM.lensed_cl(2500)

cls_mnu006eV_raw = mnu006eV.raw_cl(2500)
cls_mnu06eV_raw = mnu06eV.raw_cl(2500)
cls_LambdaCDM_raw = LambdaCDM.raw_cl(2500)
# Extract the TT power spectra
clTT_mnu006eV = cls_mnu006eV['tt'][2:]
clTT_mnu06eV = cls_mnu06eV['tt'][2:]
clTT_LambdaCDM = cls_LambdaCDM['tt'][2:]


clTT_mnu006eV_raw = cls_mnu006eV_raw['tt'][2:]
clTT_mnu06eV_raw = cls_mnu06eV_raw['tt'][2:]
clTT_LambdaCDM_raw = cls_LambdaCDM_raw['tt'][2:]

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]

# Calculate the ratios
ratio_mnu006eV_LambdaCDM = clTT_mnu006eV / clTT_LambdaCDM
ratio_mnu06eV_LambdaCDM = clTT_mnu06eV / clTT_LambdaCDM

ratio_raw_mnu006eV_LambdaCDM = clTT_mnu006eV_raw / clTT_LambdaCDM_raw
ratio_raw_mnu06eV_LambdaCDM = clTT_mnu06eV_raw / clTT_LambdaCDM_raw

# Plot
# Plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')

plt.plot(ll, ratio_mnu006eV_LambdaCDM,'b', label=r'$\sum m_\nu = 0.06$ eV (lensed)')
plt.plot(ll, ratio_mnu06eV_LambdaCDM,'r', label=r'$\sum m_\nu = 0.6$ eV (lensed)')


plt.plot(ll, ratio_raw_mnu006eV_LambdaCDM,'b--', label=r'$\sum m_\nu = 0.06$ eV (raw)')
plt.plot(ll, ratio_raw_mnu06eV_LambdaCDM,'r--', label=r'$\sum m_\nu = 0.6$ eV (raw)')

plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
plt.ylim(0.98,1.02)
plt.title("Ratio of CMB TT Power Spectra relative to $\Lambda$CDM")
plt.show()

In [ ]:
  # prompt: print the ratio of the Pks for mnu006eV and mnu06eV with respect to LambdaCDM

# get P(k) for the different models
ks_mnu006eV = np.geomspace(1e-4, 1, num=1000)
pks_mnu006eV = mnu006eV.get_pk_all(ks_mnu006eV, z=0, nonlinear=False)

ks_mnu06eV = np.geomspace(1e-4, 1, num=1000)
pks_mnu06eV = mnu06eV.get_pk_all(ks_mnu06eV, z=0, nonlinear=False)

ks_LambdaCDM = np.geomspace(1e-4, 1, num=1000)
pks_LambdaCDM = LambdaCDM.get_pk_all(ks_LambdaCDM, z=0, nonlinear=False)

# plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel("Wavenumber $k$ [1/Mpc]")
plt.ylabel(r"$P(k) / P(k)_{\Lambda\mathrm{CDM}}$")

plt.plot(ks_mnu006eV, pks_mnu006eV / pks_LambdaCDM-1, label=r'$\sum m_\nu = 0.06$ eV')
#plt.plot(ks_mnu06eV,-8 * mnu006eV.get_background()['(.)rho_ncdm[0]'][-1] / (mnu006eV.get_background()['(.)rho_cdm'][-1]+mnu06eV.get_background()['(.)rho_b'][-1])*ks_mnu06eV/ks_mnu06eV,'k--')

plt.plot(ks_mnu06eV, pks_mnu06eV / pks_LambdaCDM-1, label=r'$\sum m_\nu = 0.6$ eV')
#plt.plot(ks_mnu06eV,-8 * mnu06eV.get_background()['(.)rho_ncdm[0]'][-1] / (mnu06eV.get_background()['(.)rho_cdm'][-1]+mnu06eV.get_background()['(.)rho_b'][-1])*ks_mnu06eV/ks_mnu06eV,'k--')
plt.legend()
plt.xlim(1e-4, 1)
plt.show()

In [ ]:
# prompt: plot the data DM_over_rd normalized to the LambdaCDM prediction with their error bars

# Calculate DM/rd for LambdaCDM for the given BAO redshifts
DM_over_rd_lcdm = np.zeros_like(z_BAO)
DM_over_rd_mnu006 = np.zeros_like(z_BAO)
DM_over_rd_mnu06 = np.zeros_like(z_BAO)

for i, z in enumerate(z_BAO):
    # Calculate the comoving distance (DM)
    DM_over_rd_lcdm[i] = LambdaCDM.comoving_distance(z) / LambdaCDM.rs_drag()
    DM_over_rd_mnu006[i] = mnu006eV.comoving_distance(z) / mnu006eV.rs_drag()
    DM_over_rd_mnu06[i] = mnu06eV.comoving_distance(z) / mnu06eV.rs_drag()

# Normalize the data to the LambdaCDM prediction
normalized_DM_over_rd = DM_over_rd / DM_over_rd_lcdm
normalized_BAO_errors = BAO_errors / DM_over_rd_lcdm

plt.figure()
plt.errorbar(z_BAO, normalized_DM_over_rd, yerr=normalized_BAO_errors, fmt='o', label='BAO Data (Normalized)')
plt.plot(z_BAO, DM_over_rd_mnu006/DM_over_rd_lcdm,  label=r'$\sum m_\nu = 0.06$ eV')
plt.plot(z_BAO, DM_over_rd_mnu06/DM_over_rd_lcdm,   label=r'$\sum m_\nu = 0.6$ eV')

plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM Prediction') # Add a line at 1 for the LambdaCDM reference
plt.xlabel("Redshift z")
plt.ylabel(r"$(D_M(z)/r_d) / (D_M^{\Lambda CDM}(z)/r_d^{\Lambda CDM})$")
plt.title("BAO Data normalized to $\Lambda$CDM Prediction")
plt.legend()
plt.grid(True)
plt.show()



# Conclusions
The combination of BAO and SN1a excludes even small neutrino masses: it goes opposite to what BAO wants. Negative neutrinos?

Next:
i) play with dynamical dark energy  (w0wa) to compensate the effect of neutrinos at late-time.

ii) show the impact of the hierarchy of neutrino masses on the matter power spectrum. Can we measure it with future surveys?

# Effects of Neff on the CMB
we will compute it in 3 steps: first fixing H0, then fixing $\theta_s$, finally also fixing the matter-radiation equality redshift $1+z_{\rm eq}=\omega_m/\omega_r$



In [ ]:
# prompt: show me the effect of N_ur =3.5 and N_ur = 4 in the CMB normalized to the LambdaCDM model

# Now let's look at the impact on the CMB  for N_ur = 3.5 and N_ur = 4
common_settings=({'omega_b':0.02237,'omega_cdm':0.1200,'h':0.6736,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544,'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})
LambdaCDM = classy.Class()
LambdaCDM.set(common_settings)
LambdaCDM.compute()

# Create CLASS instances for N_ur = 3.5 and N_ur = 4
nu_35 = classy.Class()
nu_35.set(common_settings)
nu_35.set({'N_ur':3.5})
nu_35.compute()

nu_4 = classy.Class()
nu_4.set(common_settings)
nu_4.set({'N_ur':4.0})
nu_4.compute()


# Extract the CMB power spectra for each model
cls_nu_35 = nu_35.lensed_cl(2500)
cls_nu_4 = nu_4.lensed_cl(2500)
cls_LambdaCDM = LambdaCDM.lensed_cl(2500)

# Extract the TT power spectra
clTT_nu_35 = cls_nu_35['tt'][2:]
clTT_nu_4 = cls_nu_4['tt'][2:]
clTT_LambdaCDM = cls_LambdaCDM['tt'][2:]

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]


# Plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')
import matplotlib.pyplot as plt
from math import pi


# In[ ]:


# plot C_l^TT
Tcmb=2.7255*1e6
temp_factor=(Tcmb**2)
plt.figure(1)
plt.xscale('linear');plt.yscale('linear');plt.xlim(2,2500)
plt.xlabel(r'$\ell$')
plt.ylabel(r'$[\ell(\ell+1)/2\pi]  C_\ell^\mathrm{TT}$')
plt.plot(ll,clTT_LambdaCDM*ll*(ll+1)/2./pi*temp_factor,'r-')

plt.errorbar(obsdat_binned[0], obsdat_binned[1], yerr=[obsdat_binned[2], obsdat_binned[3]], fmt='.')
plt.plot(ll, clTT_nu_35*ll*(ll+1)/2./pi*temp_factor, label=r'$N_\mathrm{ur} = 3.5$')
plt.plot(ll, clTT_nu_4*ll*(ll+1)/2./pi*temp_factor, label=r'$N_\mathrm{ur} = 4.0$')


#plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM ($N_\mathrm{ur}=3.046$)') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
# plt.ylim(0.98,1.02)
plt.title(" CMB TT Power Spectra for different $N_\mathrm{ur}$")
plt.show()

In [ ]:
# prompt: show me the effect of N_ur =3.5 and N_ur = 4 in the CMB normalized to the LambdaCDM model

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]

# Calculate the ratios
ratio_nu_35_LambdaCDM = clTT_nu_35 / clTT_LambdaCDM
ratio_nu_4_LambdaCDM = clTT_nu_4 / clTT_LambdaCDM


# Plot the ratios
plt.figure()
plt.xscale('linear')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')

plt.plot(ll, ratio_nu_35_LambdaCDM-1, label=r'$N_\mathrm{ur} = 3.5$')
plt.plot(ll, ratio_nu_4_LambdaCDM-1, label=r'$N_\mathrm{ur} = 4.0$')

#plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM ($N_\mathrm{ur}=3.046$)') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
# plt.ylim(0.98,1.02)
plt.errorbar(obsdat_binned[0], (obsdat_binned[1]-obsdat_binned[4])/obsdat_binned[4], yerr=[obsdat_binned[2]/obsdat_binned[4], obsdat_binned[3]/obsdat_binned[4]], fmt='.', color='blue', ecolor='blue', capsize=3)

plt.title("Ratio of CMB TT Power Spectra relative to $\Lambda$CDM for different $N_\mathrm{ur}$")
plt.show()

In [ ]:
# prompt: show me the effect of N_ur =3.5 and N_ur = 4 in the CMB normalized to the LambdaCDM model

# Now let's look at the impact on the CMB for N_ur = 3.5 and N_ur = 4
common_settings=({'omega_b':0.02237,'omega_cdm':0.1200,'theta_s_100':1.0411,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544,'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})
LambdaCDM = classy.Class()
LambdaCDM.set(common_settings)
LambdaCDM.compute()

# Create CLASS instances for N_ur = 3.5 and N_ur = 4
nu_35 = classy.Class()
nu_35.set(common_settings)
nu_35.set({'N_ur':3.5})
nu_35.compute()

nu_4 = classy.Class()
nu_4.set(common_settings)
nu_4.set({'N_ur':4.0})
nu_4.compute()


# Extract the CMB power spectra for each model
cls_nu_35 = nu_35.lensed_cl(2500)
cls_nu_4 = nu_4.lensed_cl(2500)
cls_LambdaCDM = LambdaCDM.lensed_cl(2500)

# Extract the TT power spectra
clTT_nu_35 = cls_nu_35['tt'][2:]
clTT_nu_4 = cls_nu_4['tt'][2:]
clTT_LambdaCDM = cls_LambdaCDM['tt'][2:]

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]

# Calculate the ratios
ratio_nu_35_LambdaCDM = clTT_nu_35 / clTT_LambdaCDM
ratio_nu_4_LambdaCDM = clTT_nu_4 / clTT_LambdaCDM


# Plot the ratios
plt.figure()
plt.xscale('linear')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')

plt.plot(ll, ratio_nu_35_LambdaCDM-1, label=r'$N_\mathrm{ur} = 3.5$')
plt.plot(ll, ratio_nu_4_LambdaCDM-1, label=r'$N_\mathrm{ur} = 4.0$')

#plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM ($N_\mathrm{ur}=3.046$)') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
# plt.ylim(0.98,1.02)
plt.errorbar(obsdat_binned[0], (obsdat_binned[1]-obsdat_binned[4])/obsdat_binned[4], yerr=[obsdat_binned[2]/obsdat_binned[4], obsdat_binned[3]/obsdat_binned[4]], fmt='.', color='blue', ecolor='blue', capsize=3)

plt.title(r"Ratio of CMB TT Power Spectra relative to $\Lambda$CDM for different $N_\mathrm{ur}$ / fix $\ theta_s$ ")
plt.show()

In [ ]:
# prompt: show me the effect of N_ur =3.5 and N_ur = 4 in the CMB normalized to the LambdaCDM model

# Now let's look at the impact on the CMB for N_ur = 3.5 and N_ur = 4
common_settings=({'omega_b':0.02237,'omega_cdm':0.1200,'theta_s_100':1.0411,'A_s':2.100e-9,'n_s':0.9649,'tau_reio':0.0544,'output':'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':1.0})
LambdaCDM = classy.Class()
LambdaCDM.set(common_settings)
LambdaCDM.compute()

# Create CLASS instances for N_ur = 3.5 and N_ur = 4
nu_35 = classy.Class()
nu_35.set(common_settings)
nu_35.set({'N_ur':3.5,'omega_cdm':0.1200*(1+0.2271*3.5)/(1+0.2271*3)})
nu_35.compute()

nu_4 = classy.Class()
nu_4.set(common_settings)
nu_4.set({'N_ur':4.0,'omega_cdm':0.1200*(1+0.2271*4)/(1+0.2271*3)})
nu_4.compute()


# Extract the CMB power spectra for each model
cls_nu_35 = nu_35.lensed_cl(2500)
cls_nu_4 = nu_4.lensed_cl(2500)
cls_LambdaCDM = LambdaCDM.lensed_cl(2500)

# Extract the TT power spectra
clTT_nu_35 = cls_nu_35['tt'][2:]
clTT_nu_4 = cls_nu_4['tt'][2:]
clTT_LambdaCDM = cls_LambdaCDM['tt'][2:]

# Extract the ell values (they should be the same for all models)
ll = cls_LambdaCDM['ell'][2:]

# Calculate the ratios
ratio_nu_35_LambdaCDM = clTT_nu_35 / clTT_LambdaCDM
ratio_nu_4_LambdaCDM = clTT_nu_4 / clTT_LambdaCDM


# Plot the ratios
plt.figure()
plt.xscale('log')
plt.yscale('linear')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell^\mathrm{TT} / C_\ell^\mathrm{TT,\Lambda CDM}$')

plt.plot(ll, ratio_nu_35_LambdaCDM-1, label=r'$N_\mathrm{ur} = 3.5$')
plt.plot(ll, ratio_nu_4_LambdaCDM-1, label=r'$N_\mathrm{ur} = 4.0$')

#plt.axhline(1, color='k', linestyle='--', label=r'$\Lambda$CDM ($N_\mathrm{ur}=3.046$)') # Add a line at 1 for the LambdaCDM reference
plt.legend()
plt.xlim(2, 2500)
# plt.ylim(0.98,1.02)
plt.errorbar(obsdat_binned[0], (obsdat_binned[1]-obsdat_binned[4])/obsdat_binned[4], yerr=[obsdat_binned[2]/obsdat_binned[4], obsdat_binned[3]/obsdat_binned[4]], fmt='.', color='blue', ecolor='blue', capsize=3)

plt.title(r"Ratio of CMB TT Power Spectra relative to $\Lambda$CDM for different $N_\mathrm{ur}$ / fix $\theta_s,z_{eq}$ ")
plt.show()

The remaining effect is the "true" effect of Neff, i.e., that which cannot be absorbed by rescaling some other cosmological parameters
